## Day 49 Lecture 1 Assignment

In this assignment, we will apply GMM (Gaussian Mixture Modeling) clustering to a dataset containing player-season statistics for NBA players from the past four years.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.special import entr

This dataset contains player-season statistics for NBA players from the past four years. Each row in this dataset represents a player's per-game averages for a single season. 

This dataset contains the following variables:

- Seas: season ('2019' = 2018-2019 season, '2018' = 2017-2018 season, etc.)
- Player: player name
- Pos: position
- Age: age
- Tm: team
- G: games played
- GS: games started
- MP: minutes played
- FG: field goals
- FGA: field goals attempted
- FG%: field goal percentage
- 3P: 3 pointers
- 3PA: 3 pointers attempted
- 3P%: 3 point percentage
- 2P: 2 pointers
- 2PA: 2 pointers attempted
- 2P%: 2 point percentage
- eFG%: effective field goal percentage
- FT: free throws
- FTA: free throws attempted
- FT%: free throw percentage
- ORB: offensive rebound
- DRB: defensive rebound
- TRB: total rebounds
- AST: assists
- STL: steals
- BLK: blocks
- TOV: turnovers
- PF: personal fouls
- PTS: points

Load the dataset.

In [2]:
# answer goes here

df = pd.read_csv("https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/nba_player_seasons.csv")



In [3]:
df.head()
df.shape

(2141, 30)

The goal is to cluster these player-seasons to identify potential player "archetypes".  
The pre-processing steps will be identical to what we previously did for K-means.

Begin by removing players whose season did not meet one of the following criteria:
1. Started at least 20 games
2. Averaged at least 10 minutes per game

In [4]:
# answer goes here

df = df[(df['G']>= 20) & (df['MP'] >= 10)]



In [5]:
df.shape

(1588, 30)

Choose a subset of numeric columns that is interesting to you from an "archetypal" standpoint. 

We will choose the following basic statistics: **points, total rebounds, assists, steals, blocks**, and **turnovers**, but you should feel free to choose other reasonable feature sets if you like. Be careful not to include too many dimensions (curse of dimensionality).

In [6]:
# answer goes here

df = df[["PTS", "TRB", "AST", "STL","BLK","TOV"]]



In [7]:
df.head()

,PTS,TRB,AST,STL,BLK,TOV
0,5.3,1.5,0.6,0.5,0.2,0.5
2,3.2,1.8,1.9,0.4,0.1,0.8
3,13.9,9.5,1.6,1.5,1.0,1.7
4,8.9,7.3,2.2,0.9,0.8,1.5
7,21.3,9.2,2.4,0.5,1.3,1.8


Standardize the features in your dataset using scikit-learn's StandardScaler, which will set the mean of each feature to 0 and the variance to 1.

In [8]:
# answer goes here


scaler= StandardScaler()
scaler.fit(df)


StandardScaler(copy=True, with_mean=True, with_std=True)

Run both K-Means and Gaussian mixtures modeling twice, once with 3 cluster and once with 7 clusters. Print out the resulting means for all 4 scenarios (KM+3, GMM+3, KM+7, GMM+7). When printing the means, transform the scaled versions back into their corresponding unscaled values. 

What "archetypes" do you see? Are the archetypes identified by GMM similar to those identified by K-Means? How do the means of GMM differ from those of K-Means?

In [9]:
# answer goes here
k = 3

kmeans3 = KMeans(k)
kmeans3.fit(df)

gauss3 = GaussianMixture(k)
gauss3.fit(df)


k=7

kmeans7 = KMeans(k)
kmeans7.fit(df)

gauss7 = GaussianMixture(k)
gauss7.fit(df)

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
                means_init=None, n_components=7, n_init=1, precisions_init=None,
                random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
                verbose_interval=10, warm_start=False, weights_init=None)

In [10]:
df.columns

Index(['PTS', 'TRB', 'AST', 'STL', 'BLK', 'TOV'], dtype='object')

In [11]:
len(kmeans3.labels_)

1588

In [12]:
#centroids = scaler.inverse_transform(clst.cluster_centers_)
centers = scaler.inverse_transform(kmeans3.cluster_centers_)
kmeans3_df = pd.DataFrame(centers, columns=df.columns)


In [13]:
print(scaler.inverse_transform(gauss3.means_))
print(scaler.inverse_transform(kmeans3.cluster_centers_))

[[ 44.7836351   10.83098436   4.21390827   0.92392092   0.57906454
    1.86639002]
 [ 69.35658708  14.77428214   6.24056      1.05752062   0.65978221
    2.32383385]
 [107.10647554  19.74983625  10.16813133   1.18267889   0.83629982
    3.10727597]]
[[ 45.27332603  11.68700638   4.84353833   0.96101339   0.61695536
    1.96299312]
 [ 83.07665427  17.0234472    7.06663017   1.09042307   0.71280242
    2.54513738]
 [135.78545547  19.15644743  10.87263657   1.21375205   0.75058933
    3.33010638]]


In [14]:
print(f' K means is {kmeans3.cluster_centers_[0]}. Gauss means is {gauss3.means_[0]}')


 K means is [6.23435115 3.13773173 1.46226827 0.56575791 0.35921483 0.86782988]. Gauss means is [6.14766608 2.7802081  1.11289935 0.4720673  0.27064125 0.74183528]


In [15]:
mean_df = df.copy()

mean_df["labelk3"] = kmeans3.labels_
mean_df["labelk7"] = kmeans7.labels_
mean_df["labelg3"] = gauss3.predict(df)
mean_df["labelg7"] = gauss7.predict(df)

#prob_df = pd.DataFrame(gauss3.predict_proba(df))
#df["labelg3"] = gauss.predict(df)

In [18]:
mean_df.head()

,PTS,TRB,AST,STL,BLK,TOV,labelk3,labelk7,labelg3,labelg7
0,5.3,1.5,0.6,0.5,0.2,0.5,0,0,0,2
2,3.2,1.8,1.9,0.4,0.1,0.8,0,0,0,0
3,13.9,9.5,1.6,1.5,1.0,1.7,1,5,1,3
4,8.9,7.3,2.2,0.9,0.8,1.5,1,2,1,3
7,21.3,9.2,2.4,0.5,1.3,1.8,2,1,2,1


In [25]:
mean_df.groupby(by=['labelk3']).mean()

,PTS,TRB,AST,STL,BLK,TOV,labelk7,labelg3,labelg7
labelk3,,,,,,,,,
0,6.234351,3.137732,1.462268,0.565758,0.359215,0.867830,1.175573,0.360960,1.676118
1,12.926295,5.366534,2.695817,0.892629,0.583267,1.627092,4.621514,1.195219,2.766932
2,22.256805,6.257396,4.807692,1.204142,0.671598,2.650888,2.289941,1.810651,4.514793


In [26]:
mean_df.groupby(by=['labelk7']).mean()

,PTS,TRB,AST,STL,BLK,TOV,labelk3,labelg3,labelg7
labelk7,,,,,,,,,
0,4.900383,2.721073,1.131609,0.473946,0.320115,0.709770,0.000000,0.185824,1.657088
1,19.702400,4.673600,4.263200,1.107200,0.461600,2.272800,1.800000,1.584000,4.176000
2,8.605348,6.263636,1.533155,0.734759,0.772727,1.181818,0.427807,0.989305,2.609626
3,8.526647,2.861377,2.102994,0.698503,0.281138,1.084731,0.137725,0.497006,1.374251
4,26.246552,7.632759,5.791379,1.356897,0.853448,3.265517,2.000000,2.000000,5.500000
5,14.968333,9.005000,2.076667,0.810833,1.084167,1.772500,1.091667,1.491667,2.325000
6,13.102893,4.024380,3.342562,0.983058,0.383884,1.738017,1.000000,1.223140,3.128099


In [27]:
mean_df.groupby(by=['labelg3']).mean()

,PTS,TRB,AST,STL,BLK,TOV,labelk3,labelk7,labelg7
labelg3,,,,,,,,,
0,6.077655,2.759588,1.101268,0.467829,0.268146,0.734707,0.047544,0.969889,1.404120
1,10.573412,4.483309,2.263663,0.817578,0.464697,1.350369,0.602659,3.168390,2.388479
2,17.764643,6.615357,4.568929,1.148929,0.899643,2.438571,1.435714,3.671429,4.235714


In [28]:
mean_df.groupby(by=['labelg7']).mean()

,PTS,TRB,AST,STL,BLK,TOV,labelk3,labelk7,labelg3
labelg7,,,,,,,,,
0,7.576522,2.150725,2.053043,0.655072,0.175362,1.040290,0.179710,2.095652,0.382609
1,13.562424,7.673939,1.720000,0.635758,0.938788,1.548485,1.024242,3.909091,1.345455
2,5.525197,3.111549,0.791339,0.401050,0.308136,0.641207,0.002625,0.551181,0.020997
3,8.621479,5.028873,1.363380,0.812324,0.689789,1.043310,0.387324,2.390845,0.883803
4,12.786853,3.523506,3.903586,0.982470,0.305578,1.879283,0.888446,4.051793,1.243028
5,22.643939,4.596970,4.915152,1.356061,0.390909,2.368182,1.939394,2.242424,1.848485
6,19.354167,8.532292,5.434375,1.248958,1.088542,2.891667,1.531250,3.781250,1.979167


Predict the likelihood of each season belonging to one of the 3 clusters using the GMM model. Then, calculate the entropy for each set of predicted probabilities. 

We will use entropy as a measure of how confident we are in the predicted class label. If we had no confidence in our prediction, we would assign 33% probability to each class, while if we were totally confident, we would assign 100% to one class. Entropy would be at a maximum in the "no confidence" scenario and a minimum in the "full confidence" scenario, which makes it a reasonable way to quantify our uncertainty in our prediction. There are certainly other methods as well; feel free to experiment with them if desired.

Which five predicted labels are we least confident about? Which five are we most confident about? Print out the associated details (season, player name, stats, etc.) from those seasons.

In [17]:
# answer goes here



